# NN with dropout

In [18]:
%matplotlib inline

from commons import load_glove_embedding, pad_input, load_imdb, get_max_length, Rating, WordIndex

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

keras = tf.keras

print("Tensorflow Version: " + tf.VERSION)

Tensorflow Version: 1.13.1


In [2]:
# imdb data
imdb = load_imdb()

In [3]:
(x_train, y_train), (x_test, y_test) = imdb
word_index = WordIndex()

In [4]:
# GLOVE Word Embedding
GLOVE_DIR = "D:/google drive/haw/master/mastertheisis/hauptprojekt"
EMBEDDING_DIM = 50
embedding_index = load_glove_embedding(GLOVE_DIR, EMBEDDING_DIM)

Found 400000 word vectors.


In [5]:
(embedding_matrix, unknown_words) = word_index.match_glove(embedding_index=embedding_index, embedding_dim=EMBEDDING_DIM)

17361/88587 unknown words


In [6]:
max_length = get_max_length(x_train, x_test)

In [7]:
# pad input vectors
x_train_padded = pad_input(x_train, max_length)
x_test_padded = pad_input(x_test, max_length)
x_train_padded[0]

array([ 1, 11, 19, ...,  0,  0,  0])

In [8]:
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

embedding_layer = keras.layers.Embedding(len(word_index.index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

model = keras.Sequential([
    embedding_layer,
    keras.layers.Dropout(0.25),
    keras.layers.Conv1D(64, 15, activation="relu"),
    keras.layers.Dropout(0.25),
    keras.layers.Conv1D(64, 5, activation="relu"),
    keras.layers.MaxPooling1D(35),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2697, 50)          4429400   
_________________________________________________________________
dropout (Dropout)            (None, 2697, 50)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 2683, 64)          48064     
_________________________________________________________________
dropout_1 (Dropout)          (None, 2683, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2679, 64)          20544     
_________________________________________________________________
max_pooling1d (MaxPoo

In [9]:
tb_callback = keras.callbacks.TensorBoard(
    log_dir='./logs', 
    histogram_freq=0,
    write_graph=True, 
    write_images=True) 

model.fit(x_train_padded, y_train, validation_split=0.05, epochs=8, callbacks=[tb_callback])
loss, accuracy = model.evaluate(x_train_padded, y_train)

Train on 23750 samples, validate on 1250 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/8
23750/23750 [==============================] - 22s 936us/sample - loss: 0.5859 - acc: 0.6847 - val_loss: 0.5192 - val_acc: 0.7368
Epoch 2/8
23750/23750 [==============================] - 20s 859us/sample - loss: 0.4949 - acc: 0.7686 - val_loss: 0.4747 - val_acc: 0.7816
Epoch 3/8
23750/23750 [==============================] - 20s 863us/sample - loss: 0.4550 - acc: 0.7927 - val_loss: 0.4379 - val_acc: 0.8088
Epoch 4/8
23750/23750 [==============================] - 21s 864us/sample - loss: 0.4318 - acc: 0.8035 - val_loss: 0.4032 - val_acc: 0.8336
Epoch 5/8
23750/23750 [==============================] - 21s 863us/sample - loss: 0.4158 - acc: 0.8112 - val_loss: 0.3804 - val_acc: 0.8512
Epoch 6/8
23750/23750 [==============================] - 21s 865us/sample - loss: 0.3972 - acc: 0.8251 - val_loss: 0.3946 - val_acc: 0.8400
Epoch 7/8
23750/23750 [==============================] - 21s 88

In [10]:
def test_model(x, y):
    test_result = np.round(model.predict(x))
    test_errors = np.squeeze(test_result) != y
    correct_percentage = np.sum(test_errors) / len(y)
    print("%i / %i (%.2f%%) are correct" % (len(y) - np.sum(test_errors), len(y), 100 * (1 - correct_percentage)))

In [11]:
test_model(x_test_padded, y_test)
test_model(x_train_padded, y_train)

21017 / 25000 (84.07%) are correct
22081 / 25000 (88.32%) are correct


In [21]:
sentences = [
    "this was a very bad movie",
    "this was a very good movie",
    "I did not like this movie at all",
    "I hope there will be a sequal",
    "not bad",
    "bad",
    "not good",
    "one of the best movies of the year",
    "the first part was bad but the second part got better",
    "the first part was not bad but after that it just got worse",
    "this film was just brilliant casting location scenery story direction everyone's really suited part they played you could just imagine being there robert redford's is an amazing actor now same being director norman's father came from same scottish island as myself so i loved fact there was a real connection with this film witty remarks throughout film were great it was just brilliant so much that i bought"
]

rating = commons.Rating(word_index, model)
rating.print(rating.of(sentences, max_length))

ValueError: invalid literal for int() with base 10: '<UNK>'